In [0]:
import pandas as pd
import numpy as np
from pyspark.sql.functions import *

In [0]:
data=np.random.randint(0,100000,size=(1000000,5))

In [0]:
df = pd.DataFrame(data,columns=[f'id{str(i)}' for i in range(5)])

In [0]:
sparkdf = spark.createDataFrame(df)

In [0]:
nrow=sparkdf.count()
display(nrow)

1000000

In [0]:
f2=sparkdf.groupBy("id0").count().sort(col("count").asc()).show(5)

+-----+-----+
|  id0|count|
+-----+-----+
|47229|    1|
|87980|    1|
+-----+-----+
only showing top 2 rows



In [0]:
sparkdf.write.format("delta").mode("overwrite").saveAsTable("delta_zorder")

In [0]:
spark.sql("DESCRIBE DETAIL delta_zorder").select("sizeInBytes").show()

+-----------+
|sizeInBytes|
+-----------+
|   22475686|
+-----------+



In [0]:
%fs

ls 'dbfs:/user/hive/warehouse/delta_zorder'

path,name,size,modificationTime
dbfs:/user/hive/warehouse/delta_zorder/_delta_log/,_delta_log/,0,0
dbfs:/user/hive/warehouse/delta_zorder/part-00000-93a882b2-5433-46d6-ba3c-79bb53cb5b34-c000.snappy.parquet,part-00000-93a882b2-5433-46d6-ba3c-79bb53cb5b34-c000.snappy.parquet,2810910,1681115881000
dbfs:/user/hive/warehouse/delta_zorder/part-00001-b97391af-8785-4f69-b707-7c5e32caefb4-c000.snappy.parquet,part-00001-b97391af-8785-4f69-b707-7c5e32caefb4-c000.snappy.parquet,2811035,1681115881000
dbfs:/user/hive/warehouse/delta_zorder/part-00002-0c9baa05-3581-4182-9761-24e08146531e-c000.snappy.parquet,part-00002-0c9baa05-3581-4182-9761-24e08146531e-c000.snappy.parquet,2811369,1681115881000
dbfs:/user/hive/warehouse/delta_zorder/part-00003-ff42a22e-f369-4ca6-8b1c-eaac97b847da-c000.snappy.parquet,part-00003-ff42a22e-f369-4ca6-8b1c-eaac97b847da-c000.snappy.parquet,2810975,1681115882000
dbfs:/user/hive/warehouse/delta_zorder/part-00004-313b4aa2-1276-4ff7-8cbd-82872156ee75-c000.snappy.parquet,part-00004-313b4aa2-1276-4ff7-8cbd-82872156ee75-c000.snappy.parquet,2810889,1681115882000
dbfs:/user/hive/warehouse/delta_zorder/part-00005-ae17167b-4336-4f1b-9e2c-7aec9180d869-c000.snappy.parquet,part-00005-ae17167b-4336-4f1b-9e2c-7aec9180d869-c000.snappy.parquet,2811188,1681115882000
dbfs:/user/hive/warehouse/delta_zorder/part-00006-b8629434-1e0f-40ed-b496-038c104e4475-c000.snappy.parquet,part-00006-b8629434-1e0f-40ed-b496-038c104e4475-c000.snappy.parquet,2811635,1681115881000
dbfs:/user/hive/warehouse/delta_zorder/part-00007-c678ec50-0e49-4bb3-880a-97099d5b2b3e-c000.snappy.parquet,part-00007-c678ec50-0e49-4bb3-880a-97099d5b2b3e-c000.snappy.parquet,2811446,1681115882000


In [0]:
%fs

head 'dbfs:/user/hive/warehouse/delta_optimize_zordering/_delta_log/00000000000000000000.json'

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, LongType
def display_stats_for_delta(delta_path):
    col_values = StructType([ 
        StructField("id0",LongType(),True),
        #StructField("name",StringType(),True),
        #StructField("heart_rate",LongType(),True)
      ])
    schema = StructType([ 
        StructField("numRecords", LongType(),True),
        StructField("minValues", col_values, True),
        StructField("maxValues", col_values, True)
      ])
    (spark.read.json(delta_path)
        .select("add")
        .where("add is not null")
        .selectExpr("add.path", "add.size", "add.stats")
        .withColumn("stats", F.from_json(F.col("stats"), schema))
        .withColumn("numRecords", F.expr("stats.numRecords"))
        .withColumn("min_id", F.expr("stats.minValues.id0"))
        .withColumn("max_id", F.expr("stats.maxValues.id0"))
        .drop("stats")
        .orderBy("path")
        .display()
    )

In [0]:
display_stats_for_delta("dbfs:/user/hive/warehouse/delta_zorder/_delta_log/00000000000000000000.json")

path,size,numRecords,min_id,max_id
part-00000-93a882b2-5433-46d6-ba3c-79bb53cb5b34-c000.snappy.parquet,2810910,125000,0,99999
part-00001-b97391af-8785-4f69-b707-7c5e32caefb4-c000.snappy.parquet,2811035,125000,0,99999
part-00002-0c9baa05-3581-4182-9761-24e08146531e-c000.snappy.parquet,2811369,125000,0,99999
part-00003-ff42a22e-f369-4ca6-8b1c-eaac97b847da-c000.snappy.parquet,2810975,125000,0,99997
part-00004-313b4aa2-1276-4ff7-8cbd-82872156ee75-c000.snappy.parquet,2810889,125000,0,99999
part-00005-ae17167b-4336-4f1b-9e2c-7aec9180d869-c000.snappy.parquet,2811188,125000,2,99997
part-00006-b8629434-1e0f-40ed-b496-038c104e4475-c000.snappy.parquet,2811635,125000,1,99998
part-00007-c678ec50-0e49-4bb3-880a-97099d5b2b3e-c000.snappy.parquet,2811446,125000,1,99999


In [0]:
%sql

select * from delta_zorder where id0 = 87980

id0,id1,id2,id3,id4
87980,36472,30105,92095,40770


In [0]:
spark.table("delta_zorder").repartition(8, "id0").write.mode("overwrite").saveAsTable("repartition")
display_stats_for_delta("dbfs:/user/hive/warehouse/repartition/_delta_log/00000000000000000000.json")


path,size,numRecords,min_id,max_id
part-00000-8d6252f7-975a-440d-80c0-f060c0bd8137-c000.snappy.parquet,2536903,125596,2,99999
part-00001-44f8655f-c766-4cf6-8c78-9c3f90452391-c000.snappy.parquet,2502818,123861,9,99989
part-00002-318be3c0-9cd6-49eb-a689-50d09c86c18e-c000.snappy.parquet,2525239,124985,14,99997
part-00003-af58089f-bf52-4474-b67c-0376a6a32f39-c000.snappy.parquet,2540927,125743,3,99996
part-00004-da15bede-0711-476b-ae5a-b9917d4ebe54-c000.snappy.parquet,2503462,123909,4,99998
part-00005-37dde635-6444-4dd3-8649-86af00c3214c-c000.snappy.parquet,2518635,124651,0,99995
part-00006-544a36c4-7c2b-45b1-b12b-9fcf03bbbc96-c000.snappy.parquet,2517066,124617,16,99988
part-00007-002ac8ab-358a-4f73-b0c6-5e4276e4e9be-c000.snappy.parquet,2558877,126638,13,99992


In [0]:
%sql
select * from repartition where id0 = 87980

id0,id1,id2,id3,id4
87980,36472,30105,92095,40770


In [0]:
spark.table("delta_zorder").repartitionByRange(8, "id0").write.mode("overwrite").saveAsTable("rangepartition_by_id")
display_stats_for_delta("dbfs:/user/hive/warehouse/rangepartition_by_id/_delta_log/00000000000000000000.json")

path,size,numRecords,min_id,max_id
part-00000-55a2956b-e28f-4495-a9ee-420763842a43-c000.snappy.parquet,2571513,127608,0,12807
part-00001-c8ed45fc-e963-4daf-a865-56b49db29914-c000.snappy.parquet,2633741,130727,12808,25828
part-00002-0fb2c9b8-4bb5-4bfe-8b54-e33d05a024c1-c000.snappy.parquet,2817309,139839,25829,39830
part-00003-d21ed507-9651-4360-bd55-0143278be8c2-c000.snappy.parquet,2294640,113860,39831,51243
part-00004-c22584fa-0211-449f-9624-fd3d1cb0f49d-c000.snappy.parquet,2462577,122197,51244,63484
part-00005-a6a92773-e244-4fb3-8221-b011a864fbe0-c000.snappy.parquet,2545574,126333,63485,76121
part-00006-c7d9e4be-1ab8-4c5f-ad85-70e16d2e4952-c000.snappy.parquet,2469994,122579,76122,88343
part-00007-5a9ddc92-0b7f-47a6-9959-134c6d337e96-c000.snappy.parquet,2354600,116857,88344,99999


In [0]:
%sql
select * from rangepartition_by_id where id0 = 87980

id0,id1,id2,id3,id4
87980,36472,30105,92095,40770


In [0]:
%sql

OPTIMIZE delta_zorder ZORDER BY (id0)

path,metrics
dbfs:/user/hive/warehouse/delta_zorder,"List(1, 8, List(22475686, 22475686, 2.2475686E7, 1, 22475686), List(2810889, 2811635, 2811180.875, 8, 22489447), 0, List(minCubeSize(107374182400), List(0, 0), List(8, 22489447), 0, List(8, 22489447), 1, null), 1, 8, 0, false, 0, 0, 1681116993343, 1681117006154, 8, 1, null)"


In [0]:
%fs

head 'dbfs:/user/hive/warehouse/delta_zorder/_delta_log/00000000000000000001.json'

{"commitInfo":{"timestamp":1681117003000,"userId":"3818739110978553","userName":"venkatkarthick15@outlook.com","operation":"OPTIMIZE","operationParameters":{"predicate":"[]","zOrderBy":"[\"id0\"]","batchId":"0","auto":false},"notebook":{"notebookId":"4479115915243899"},"clusterId":"0410-082958-s3yy2l5p","readVersion":0,"isolationLevel":"SnapshotIsolation","isBlindAppend":false,"operationMetrics":{"numRemovedFiles":"8","numRemovedBytes":"22489447","p25FileSize":"22475686","minFileSize":"22475686","numAddedFiles":"1","maxFileSize":"22475686","p75FileSize":"22475686","p50FileSize":"22475686","numAddedBytes":"22475686"},"engineInfo":"Databricks-Runtime/11.3.x-scala2.12","txnId":"0fe1761e-0ec3-48e5-aaeb-b3e4175e7325"}}
{"remove":{"path":"part-00000-93a882b2-5433-46d6-ba3c-79bb53cb5b34-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2810910,"tags":{"MAX_INSERTION_TIME":"1681115881000000","INSERTION_TIME":"1681115881000000","MIN_INSERTION_TIME":"1681115881000000","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00001-b97391af-8785-4f69-b707-7c5e32caefb4-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2811035,"tags":{"MAX_INSERTION_TIME":"1681115881000001","INSERTION_TIME":"1681115881000001","MIN_INSERTION_TIME":"1681115881000001","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00002-0c9baa05-3581-4182-9761-24e08146531e-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2811369,"tags":{"MAX_INSERTION_TIME":"1681115881000002","INSERTION_TIME":"1681115881000002","MIN_INSERTION_TIME":"1681115881000002","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00003-ff42a22e-f369-4ca6-8b1c-eaac97b847da-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2810975,"tags":{"MAX_INSERTION_TIME":"1681115881000003","INSERTION_TIME":"1681115881000003","MIN_INSERTION_TIME":"1681115881000003","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00004-313b4aa2-1276-4ff7-8cbd-82872156ee75-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2810889,"tags":{"MAX_INSERTION_TIME":"1681115881000004","INSERTION_TIME":"1681115881000004","MIN_INSERTION_TIME":"1681115881000004","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00005-ae17167b-4336-4f1b-9e2c-7aec9180d869-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2811188,"tags":{"MAX_INSERTION_TIME":"1681115881000005","INSERTION_TIME":"1681115881000005","MIN_INSERTION_TIME":"1681115881000005","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00006-b8629434-1e0f-40ed-b496-038c104e4475-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2811635,"tags":{"MAX_INSERTION_TIME":"1681115881000006","INSERTION_TIME":"1681115881000006","MIN_INSERTION_TIME":"1681115881000006","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"remove":{"path":"part-00007-c678ec50-0e49-4bb3-880a-97099d5b2b3e-c000.snappy.parquet","deletionTimestamp":1681116996906,"dataChange":false,"extendedFileMetadata":true,"partitionValues":{},"size":2811446,"tags":{"MAX_INSERTION_TIME":"1681115881000007","INSERTION_TIME":"1681115881000007","MIN_INSERTION_TIME":"1681115881000007","OPTIMIZE_TARGET_SIZE":"268435456"}}}
{"add":{"path":"part-00000-7fa38590-67a1-4a69-ab3b-c41856b5a48a-c000.snappy.parquet","partitionValues":{},"size":22475686,"modificationTime":1681117003000,"dataChange":false,"stats":"{\"numRecords\":1000000,\"minValues\":{\"id0\":0,\"id1\":0,\"id2\":0,\"id3\":0,\"id4\":0},\"maxValues\":{\"id0\":99999,\"id1\":99999,\"id2\":99999,\"id3\":99

In [0]:
spark.sql("DESCRIBE HISTORY delta_zorder").select("version","timestamp").show()

+-------+-------------------+
|version|          timestamp|
+-------+-------------------+
|      1|2023-04-10 08:56:44|
|      0|2023-04-10 08:38:05|
+-------+-------------------+



In [0]:
display_stats_for_delta("dbfs:/user/hive/warehouse/delta_zorder/_delta_log/00000000000000000001.json")

path,size,numRecords,min_id,max_id
part-00000-7fa38590-67a1-4a69-ab3b-c41856b5a48a-c000.snappy.parquet,22475686,1000000,0,99999


In [0]:
%sql
select * from delta_zorder where id0 = 87980

id0,id1,id2,id3,id4
87980,36472,30105,92095,40770
